In [42]:
# import geopandas as gpd
# import geopy
# from shapely.geometry import Point
# from geopy.geocoders import Nominatim

import pandas as pd
import numpy as np
import mpu
import seaborn as sns

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)


In [43]:
df = pd.read_excel('atm_bank_misr.xlsx')
#df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4263 entries, 0 to 4262
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           4263 non-null   int64  
 1   governerate  4263 non-null   object 
 2   region       4263 non-null   object 
 3   Address      4263 non-null   object 
 4   Lat          4263 non-null   float64
 5   Long         4263 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 200.0+ KB


In [44]:
df.head()

ID governerate            region                                        Address        Lat       Long
0   1   Al Fayyum  Markaz Al-Fayyum                  Alfayoum Beghous Alhoriea St.  29.304412  30.853546
1   2   Al Fayyum  Markaz Al-Fayyum                              Fayoum University  29.319696  30.835641
2   3   Al Fayyum  Markaz Al-Fayyum                            Fayoum AlHoriea St.  29.308199  30.842229
3   4   Al Fayyum   Markaz Sinnuris                               Sanoras - Fayoum  29.408575  30.865073
4   5   Al Fayyum       Markaz Itsa  194 Abo Bakr AlSiddiq St.- Atsa City - Fayoum  29.236966  30.790194

In [45]:
df.iloc[[55,132]]

ID  governerate             region                     Address        Lat      Long
55    56  Al Buhayrah  Markaz Abu Hummus  The local unit - Abo Homos  31.100602  30.31555
132  133  Al Buhayrah  Markaz Abu Hummus  The local unit - Abo Homos  31.100602  30.31555

In [46]:
lat_longs = np.array(df[['Lat','Long']])#

In [47]:
df

ID       governerate                                             region                                            Address        Lat       Long
0        1         Al Fayyum                                   Markaz Al-Fayyum                      Alfayoum Beghous Alhoriea St.  29.304412  30.853546
1        2         Al Fayyum                                   Markaz Al-Fayyum                                  Fayoum University  29.319696  30.835641
2        3         Al Fayyum                                   Markaz Al-Fayyum                                Fayoum AlHoriea St.  29.308199  30.842229
3        4         Al Fayyum                                    Markaz Sinnuris                                   Sanoras - Fayoum  29.408575  30.865073
4        5         Al Fayyum                                        Markaz Itsa      194 Abo Bakr AlSiddiq St.- Atsa City - Fayoum  29.236966  30.790194
5        6         Al Fayyum                                   Markaz Al-Fayyum                   AlFayoum 45 AlHorreya St. Fayoum  29.307723  30.846140
6        7         Al Fayyum                                   Markaz Al-Fayyum     Gamal Abd AlNasser St. - AlMasalla Sq.- Fayoum  29.315420  30.852179
7        8         Al Fayyum                                   Markaz Al-Fayyum               10th of Ramadan - Industrial Zone B1  29.321487  30.837896
8        9         Al Fayyum                                   Markaz Al-Fayyum                  Faculty of Arts Fayoum University  29.322145  30.837087
9       10         Al Fayyum                                   Markaz Al-Fayyum                               Fayoum train station  29.308889  30.847499
10      11         Al Fayyum                                   Markaz Al-Fayyum                          Batal Alsalam St.- Fayoum  29.304818  30.843850
11      12         Al Fayyum                                    Markaz Ibshaway                            AlGomhoria St. Ibshaway  29.370735  30.688374
12      13         Al Fayyum                                   Markaz Al-Fayyum                                  Saad Zaghloul St.  29.319011  30.851452
13      14         Al Fayyum                                    Markaz Ibshaway  Algomhoria St. in front of Ibshaway Fayoum Pos...  29.360741  30.683258
14      15         Al Fayyum                                   Markaz Al-Fayyum  B 32 Almasalah Sq. Faioum Bandar Alfaioum Gove...  29.315635  30.852531
15      16         Al Fayyum                                     Markaz Tamiyah          23rd of July St. - Tamiya Center - Fayoum  29.474276  30.954961
16      17         Al Fayyum                                    Markaz Sinnuris              Almadares St. Sanoras Sector Alfaioum  29.408703  30.858720
17      18         Al Fayyum                                   Markaz Al-Fayyum  Jamal Abdel Nasser Street, behind the local po...  29.309736  30.847641
18      19         Al Fayyum                                    Markaz Sinnuris            Tersa Village - Senoras Center - Fayoum  29.417200  30.817362
19      20         Al Fayyum                                    Markaz Sinnuris         Vidimin Village - Senoures Center - Fayoum  29.381256  30.777553
20      21         Al Fayyum                                    Markaz Sinnuris         Naqalifa Village - Senoras Sector - Fayoum  29.396940  30.824684
21      22         Al Fayyum                                   Markaz Al-Fayyum    12 Khaled Bin Al Waleed Street, Baghous, Fayoum  29.302993  30.852963
22      23         Al Fayyum                                        Markaz Itsa  Governor of Fayoum - Abu Jandir - Atsa Center ...  29.241247  30.672294
23      24         Al Fayyum                                   Markaz Al-Fayyum  Hospital Street next to Lahoun Hospital and Ag...  29.209594  30.971382
24      25         Al Fayyum                                    Markaz Sinnuris  Fayoum - in front of the Society for the Prese...  29.407404  30.770053
25      26         Al Fayyum  

In [ ]:
k = 4


near_dis1 = [] # distance of nearest atm
index1 = [] # index of nearest atm

near_dis2 = []
index2 = []

near_dis3 = []
index3 = []

for i in range(len(lat_longs)):
    err_list = []
    
    for j in range(len(lat_longs)): 
        
        lat1,lon1,lat2,lon2 = lat_longs[i][0],lat_longs[i][1],lat_longs[j][0],lat_longs[j][1]
        err_list.append(mpu.haversine_distance((lat1, lon1), (lat2, lon2)))
    
    distances = np.array(err_list)
    least_k = np.argsort(distances)[1:k]
    print('Indexes of nearest lat_long',least_k)
    print('For lat_long',lat_longs[i],'the nearest lat_longs are:')
    print(lat_longs[least_k])
    print('With distances of:') 
    print(distances[least_k])
    
    least_k_distances = distances[least_k]
    near_dis1.append(least_k_distances[0])
    index1.append(least_k[0])
    near_dis2.append(least_k_distances[1])
    index2.append(least_k[1])
    near_dis3.append(least_k_distances[2])
    index3.append(least_k[2])
    print('-------------------------------------')
    
    
    
    
    

Indexes of nearest lat_long [21 29  9]
For lat_long [29.30441208 30.85354563] the nearest lat_longs are:
[[29.3029927  30.85296321]
 [29.308889   30.847499  ]
 [29.308889   30.847499  ]]
With distances of:
[0.1676278  0.76913315 0.76913315]
-------------------------------------
Indexes of nearest lat_long [32  7 31]
For lat_long [29.319696 30.835641] the nearest lat_longs are:
[[29.319696 30.835641]
 [29.321487 30.837896]
 [29.322145 30.837087]]
With distances of:
[0.         0.29573057 0.30628301]
-------------------------------------
Indexes of nearest lat_long [ 2  5 35]
For lat_long [29.308199 30.842229] the nearest lat_longs are:
[[29.308199   30.842229  ]
 [29.30772267 30.84613982]
 [29.304818   30.84385   ]]
With distances of:
[0.         0.38288298 0.40748412]
-------------------------------------
Indexes of nearest lat_long [16 25 20]
For lat_long [29.40857519 30.86507307] the nearest lat_longs are:
[[29.4087034  30.8587204 ]
 [29.41540081 30.86429428]
 [29.39694049 30.8246841

Indexes of nearest lat_long [1289 1288 1310]
For lat_long [29.07578529 31.09938927] the nearest lat_longs are:
[[29.07578529 31.09938927]
 [29.07523208 31.09924149]
 [29.07535899 31.09829712]]
With distances of:
[5.31824916e-07 6.31679329e-02 1.16241722e-01]
-------------------------------------
Indexes of nearest lat_long [ 190   43 3350]
For lat_long [31.10691837 30.64172916] the nearest lat_longs are:
[[31.10958367 30.63910172]
 [31.103395   30.638362  ]
 [31.12534257 30.64901408]]
With distances of:
[0.38782124 0.50622396 2.16287267]
-------------------------------------
Indexes of nearest lat_long [ 46 153  84]
For lat_long [30.9135036  30.29365764] the nearest lat_longs are:
[[30.912765 30.295561]
 [30.912765 30.295561]
 [30.915766 30.297315]]
With distances of:
[0.19928897 0.19928897 0.43013985]
-------------------------------------
Indexes of nearest lat_long [ 40 177 128]
For lat_long [31.024659 30.607641] the nearest lat_longs are:
[[31.024659   30.607641  ]
 [31.02393382 30.

Indexes of nearest lat_long [ 83  44 145]
For lat_long [30.88483388 30.66364617] the nearest lat_longs are:
[[30.884864 30.665297]
 [30.886466 30.662967]
 [30.886466 30.662967]]
With distances of:
[0.15757038 0.19270895 0.19270895]
-------------------------------------
Indexes of nearest lat_long [114 193 192]
For lat_long [30.91078908 30.17701905] the nearest lat_longs are:
[[30.91158881 30.17717889]
 [30.91144225 30.17490715]
 [30.91290404 30.17689168]]
With distances of:
[0.09022356 0.21416911 0.2354862 ]
-------------------------------------
Indexes of nearest lat_long [ 75 119 106]
For lat_long [31.03450281 30.47469798] the nearest lat_longs are:
[[31.03449464 30.47471262]
 [31.03428061 30.46823018]
 [31.041921   30.473102  ]]
With distances of:
[0.00166475 0.61673596 0.83876286]
-------------------------------------
Indexes of nearest lat_long [ 74 119 106]
For lat_long [31.03449464 30.47471262] the nearest lat_longs are:
[[31.03450281 30.47469798]
 [31.03428061 30.46823018]
 [31

Indexes of nearest lat_long [ 41 116 198]
For lat_long [31.1824926  30.52713284] the nearest lat_longs are:
[[31.18275058 30.52755669]
 [31.1838385  30.52745446]
 [31.18416312 30.52710423]]
With distances of:
[0.04948385 0.15275265 0.18577329]
-------------------------------------
Indexes of nearest lat_long [129 200  76]
For lat_long [31.04438807 30.4667885 ] the nearest lat_longs are:
[[31.0463062  30.4652941 ]
 [31.046464   30.46865632]
 [31.04241948 30.46890924]]
With distances of:
[0.25643624 0.291457   0.29788686]
-------------------------------------
Indexes of nearest lat_long [113  48 161]
For lat_long [30.8601142  30.15493118] the nearest lat_longs are:
[[30.9063534 30.1736873]
 [30.912211  30.170589 ]
 [30.912211  30.170589 ]]
With distances of:
[5.44420412 5.9824931  5.9824931 ]
-------------------------------------
Indexes of nearest lat_long [193  73 114]
For lat_long [30.9063534 30.1736873] the nearest lat_longs are:
[[30.91144225 30.17490715]
 [30.91078908 30.17701905]


Indexes of nearest lat_long [ 40 177 128]
For lat_long [31.024659 30.607641] the nearest lat_longs are:
[[31.024659   30.607641  ]
 [31.02393382 30.60350721]
 [31.013742   30.57017929]]
With distances of:
[0.         0.40207097 3.7706048 ]
-------------------------------------
Indexes of nearest lat_long [153  39 191]
For lat_long [30.912765 30.295561] the nearest lat_longs are:
[[30.912765   30.295561  ]
 [30.9135036  30.29365764]
 [30.91038848 30.29732155]]
With distances of:
[0.         0.19928897 0.31311606]
-------------------------------------
Indexes of nearest lat_long [154  95 155]
For lat_long [31.041921 30.473102] the nearest lat_longs are:
[[31.041921 30.473102]
 [31.04408  30.475274]
 [31.04408  30.475274]]
With distances of:
[0.         0.31694127 0.31694127]
-------------------------------------
Indexes of nearest lat_long [ 95 106 154]
For lat_long [31.04408  30.475274] the nearest lat_longs are:
[[31.04408  30.475274]
 [31.041921 30.473102]
 [31.041921 30.473102]]
With

...

Indexes of nearest lat_long [1780 1426 2013]
For lat_long [30.113699 31.395402] the nearest lat_longs are:
[[30.113739 31.395544]
 [30.11411  31.396824]
 [30.115871 31.393739]]
With distances of:
[0.01436452 0.1442108  0.28968267]
-------------------------------------
Indexes of nearest lat_long [1779 1426 2013]
For lat_long [30.113739 31.395544] the nearest lat_longs are:
[[30.113699 31.395402]
 [30.11411  31.396824]
 [30.115871 31.393739]]
With distances of:
[0.01436452 0.12984675 0.29384262]
-------------------------------------
Indexes of nearest lat_long [1404 1561 2186]
For lat_long [30.091456 31.33637 ] the nearest lat_longs are:
[[30.09010356 31.34114395]
 [30.094296   31.344778  ]
 [30.091713   31.326371  ]]
With distances of:
[0.48329187 0.8683684  0.96241454]
-------------------------------------
Indexes of nearest lat_long [1782 1370 1645]
For lat_long [30.040114 31.250926] the nearest lat_longs are:
[[30.040114   31.250926  ]
 [30.03775884 31.24960743]
 [30.03636734 31.251

Indexes of nearest lat_long [2081 2064 1815]
For lat_long [30.071591 31.288335] the nearest lat_longs are:
[[30.071591 31.288335]
 [30.071543 31.288413]
 [30.071543 31.288413]]
With distances of:
[0.         0.00921001 0.00921001]
-------------------------------------
Indexes of nearest lat_long [1815 2081 1814]
For lat_long [30.071543 31.288413] the nearest lat_longs are:
[[30.071543 31.288413]
 [30.071591 31.288335]
 [30.071591 31.288335]]
With distances of:
[0.         0.00921001 0.00921001]
-------------------------------------
Indexes of nearest lat_long [1894 1817 2049]
For lat_long [30.066363 31.285149] the nearest lat_longs are:
[[30.066363 31.285149]
 [30.066387 31.285174]
 [30.066387 31.285174]]
With distances of:
[0.         0.00359303 0.00359303]
-------------------------------------
Indexes of nearest lat_long [2049 1816 1894]
For lat_long [30.066387 31.285174] the nearest lat_longs are:
[[30.066387 31.285174]
 [30.066363 31.285149]
 [30.066363 31.285149]]
With distances o

Indexes of nearest lat_long [2000 1562 1384]
For lat_long [30.068397 31.319048] the nearest lat_longs are:
[[30.06582717 31.31976267]
 [30.07038    31.321645  ]
 [30.066072   31.323309  ]]
With distances of:
[0.29391183 0.33327917 0.48474168]
-------------------------------------
Indexes of nearest lat_long [2262 1392 1890]
For lat_long [30.107845 31.348148] the nearest lat_longs are:
[[30.107845   31.348148  ]
 [30.10880038 31.35015039]
 [30.10511334 31.35012904]]
With distances of:
[0.         0.21996791 0.35857619]
-------------------------------------
Indexes of nearest lat_long [1432 1478 2091]
For lat_long [29.964489 31.263617] the nearest lat_longs are:
[[29.96644034 31.26888189]
 [29.965403   31.26983   ]
 [29.965403   31.26983   ]]
With distances of:
[0.55163797 0.60707611 0.60707611]
-------------------------------------
Indexes of nearest lat_long [1856 2147 2149]
For lat_long [30.084289 31.292315] the nearest lat_longs are:
[[30.084289   31.292315  ]
 [30.08541292 31.294329

Indexes of nearest lat_long [1540 2347 1870]
For lat_long [30.12597829 31.33127184] the nearest lat_longs are:
[[30.126541 31.330057]
 [30.126541 31.330057]
 [30.123735 31.326028]]
With distances of:
[0.13253669 0.13253669 0.56264835]
-------------------------------------
Indexes of nearest lat_long [1894 1817 2049]
For lat_long [30.066363 31.285149] the nearest lat_longs are:
[[30.066363 31.285149]
 [30.066387 31.285174]
 [30.066387 31.285174]]
With distances of:
[0.         0.00359303 0.00359303]
-------------------------------------
Indexes of nearest lat_long [2176 1618 1949]
For lat_long [30.05758644 31.50922046] the nearest lat_longs are:
[[30.05754456 31.50864427]
 [30.05009922 31.51482295]
 [30.06662755 31.49710406]]
With distances of:
[0.05564871 0.99190523 1.53959457]
-------------------------------------
Indexes of nearest lat_long [2189 1719 1560]
For lat_long [30.07441948 31.30118531] the nearest lat_longs are:
[[30.073239   31.300957  ]
 [30.073239   31.300957  ]
 [30.076

Indexes of nearest lat_long [1761 2100 1845]
For lat_long [30.00621817 31.42522886] the nearest lat_longs are:
[[30.006031 31.426481]
 [30.006031 31.426481]
 [30.004308 31.424826]]
With distances of:
[0.12235376 0.12235376 0.21591463]
-------------------------------------
Indexes of nearest lat_long [2057 1665 1413]
For lat_long [30.0277774  31.40709366] the nearest lat_longs are:
[[30.029769 31.407076]
 [30.029769 31.407076]
 [30.029479 31.410052]]
With distances of:
[0.22146234 0.22146234 0.34192189]
-------------------------------------
Indexes of nearest lat_long [641 645 754]
For lat_long [30.2136625  31.68213934] the nearest lat_longs are:
[[30.241871   31.726754  ]
 [30.24187143 31.72675367]
 [30.245731   31.731391  ]]
With distances of:
[5.31146644 5.31146909 5.92497018]
-------------------------------------
Indexes of nearest lat_long [2151 1456 2066]
For lat_long [30.02558063 31.45554631] the nearest lat_longs are:
[[30.03029331 31.45698432]
 [30.020047   31.466243  ]
 [30.02

In [ ]:
nearest_df1 = df.iloc[index1]
nearest_df1

In [ ]:
nearest_df1

In [ ]:
#Id1 | address1 | gov1 | section1 | lat1 | long1 | ---- | Id2 | address | gov | section | lat | long  | Id2 | address | gov | section | lat | long | Id2 | address | gov | section | lat | long

In [ ]:
new_df = pd.concat([df.reset_index(drop=True),nearest_df1.reset_index(drop=True)], axis=1)

In [ ]:
dummy = [round(x, 3) for x in near_dis1]


In [ ]:
new_df['distance1'] = dummy
new_df['distance_in_meters'] = new_df['distance1'] * 1000
new_df

In [ ]:
new_df.to_excel('nearest_atm2.xlsx')

In [ ]:
sns.boxplot(new_df['distance1'])

In [ ]:
new_df[new_df['distance1']==0].count()

In [ ]:
# model to measure wether the number of the 
# atms satisfies soem sort of model(equation) based on the parameters like 
# population number and where are the atms in some area